## http://destinydevs.github.io/BungieNetPlatform/docs/Manifest
## Following the python code here for getting and using the manifest
It's for D1 but I guess it's actually the same for d2.  We're working through it.

In [1]:
import requests
import os
import zipfile
import pickle
import json
import sqlite3

from basics import *
from enums import *

In [2]:
mnfst = make_request("destiny2/manifest")

In [3]:
dir(mnfst)[-7:]

['data', 'error_code', 'error_status', 'exception', 'message', 'status', 'url']

In [4]:
mnfst.data['mobileWorldContentPaths']['en'] # dictionary path to .content file (zipped SQLlite or something)

'/common/destiny2_content/sqlite/en/world_sql_content_c41e799c128f0c26d2fea1676d9ee8b7.content'

In [5]:
manifest_url = "http://www.bungie.net"+mnfst.data['mobileWorldContentPaths']['en']
manifest_url  # url to download manifest!

'http://www.bungie.net/common/destiny2_content/sqlite/en/world_sql_content_c41e799c128f0c26d2fea1676d9ee8b7.content'

In [6]:
# now let's get it
r = requests.get(manifest_url)
with open("MANZIP", "wb") as zip:
    zip.write(r.content)
print("manifest download done")

manifest download done


In [7]:
# now unzip it

if os.path.exists('Manifest.content'):
    os.remove('Manifest.content')

with zipfile.ZipFile('MANZIP') as zp:
    name = zp.namelist()
    zp.extractall()

os.rename(name[0], "Manifest.content")
print("Unzipped")


Unzipped


In [8]:
# let me try just doing some basic queries
# set up connection and cursor
con = sqlite3.connect('manifest.content')
cur = con.cursor()

def fetch():
    for item in cur.fetchall():
        print(item, "\n")

In [9]:
# get all tables from which we can query
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())

[('DestinyPlaceDefinition',), ('DestinyActivityDefinition',), ('DestinyActivityTypeDefinition',), ('DestinyClassDefinition',), ('DestinyGenderDefinition',), ('DestinyInventoryBucketDefinition',), ('DestinyRaceDefinition',), ('DestinyTalentGridDefinition',), ('DestinyUnlockDefinition',), ('DestinyMaterialRequirementSetDefinition',), ('DestinySandboxPerkDefinition',), ('DestinyStatGroupDefinition',), ('DestinyFactionDefinition',), ('DestinyVendorGroupDefinition',), ('DestinyRewardSourceDefinition',), ('DestinyItemCategoryDefinition',), ('DestinyDamageTypeDefinition',), ('DestinyActivityModeDefinition',), ('DestinyMedalTierDefinition',), ('DestinyAchievementDefinition',), ('DestinyActivityGraphDefinition',), ('DestinyBondDefinition',), ('DestinyCollectibleDefinition',), ('DestinyDestinationDefinition',), ('DestinyEquipmentSlotDefinition',), ('DestinyStatDefinition',), ('DestinyInventoryItemDefinition',), ('DestinyItemTierTypeDefinition',), ('DestinyLocationDefinition',), ('DestinyLoreDefi

In [10]:
cur.execute("""SELECT * FROM DestinyActivityDefinition LIMIT 5 """)
#fetch()
[d[0] for d in cur.description]  # get column names

['id', 'json']

In [11]:
# original version from site
# I think the hash dict is just irrelevant.  From what I've seen it's all just 'hash' now.  Let's try in a seperate block.
"""
def build_dict(hash_dict):
    #connect to the manifest
    con = sqlite3.connect('manifest.content')
    print('Connected')
    #create a cursor object
    cur = con.cursor()

    all_data = {}
    #for every table name in the dictionary
    for table_name in hash_dict.keys():
        #get a list of all the jsons from the table
        cur.execute('SELECT json from '+table_name)
        print('Generating '+table_name+' dictionary....')

        #this returns a list of tuples: the first item in each tuple is our json
        items = cur.fetchall()

        #create a list of jsons
        item_jsons = [json.loads(item[0]) for item in items]

        #create a dictionary with the hashes as keys
        #and the jsons as values
        item_dict = {}
        hash = hash_dict[table_name]
        for item in item_jsons:
            item_dict[item[hash]] = item

        #add that dictionary to our all_data using the name of the table
        #as a key.
        all_data[table_name] = item_dict

    print('Dictionary Generated!')
    return all_data
#check if pickle exists, if not create one.
if os.path.isfile(r'path\to\file\manifest.content') == False:
    get_manifest()
    all_data = build_dict(hashes)
    with open('manifest.pickle', 'wb') as data:
        pickle.dump(all_data, data)
        print("'manifest.pickle' created!\nDONE!")
else:
    print('Pickle Exists')

with open('manifest.pickle', 'rb') as data:
    all_data = pickle.load(data)
"""

print()

In [12]:
def build_dict():
    #connect to the manifest
    con = sqlite3.connect('manifest.content')
    print('Connected')
    #create a cursor object
    cur = con.cursor()
    
    table_list = get_table_list()

    all_data = {}
    #for every table name in the dictionary
    for table_name in table_list:
        #get a list of all the jsons from the table
        cur.execute('SELECT json from '+table_name)
        print('Generating '+table_name+' dictionary....')

        #this returns a list of tuples: the first item in each tuple is our json
        items = cur.fetchall()

        #create a list of jsons
        item_jsons = [json.loads(item[0]) for item in items]

        #create a dictionary with the hashes as keys
        #and the jsons as values
        item_dict = {}
        #hash = hash_dict[table_name]
        try:
            for item in item_jsons:
                item_dict[item["hash"]] = item
                
        except KeyError:
            # for some reason this table uses "statId" not "hash"
            # too lazy to hard code for one edge case so we'll do this
            if table_name == "DestinyHistoricalStatsDefinition":
                for item in item_jsons:
                    item_dict[item["statId"]] = item
            
            else:
                print("Table has no hash and also is not HistoricalStats -max")

        #add that dictionary to our all_data using the name of the table
        #as a key.
        all_data[table_name] = item_dict

    print('Dictionary Generated!')
    return all_data

def get_table_list():
    """
    Replacement for hash_dict since I believe it may be irrelevant
    Just a list of all tables
    """
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tbls = [tbl[0] for tbl in cur.fetchall()] # get list of tables
    return tbls

#check if pickle exists, if not create one.
if os.path.isfile(r'path\to\file\manifest.content') == False:
    #get_manifest() # -- did not put in a function, it's already been run
    all_data = build_dict() #build_dict(hashes)
    with open('manifest.pickle', 'wb') as data:
        pickle.dump(all_data, data)
        print("'manifest.pickle' created!\nDONE!")
else:
    print('Pickle Exists')

with open('manifest.pickle', 'rb') as data:
    all_data = pickle.load(data)

Connected
Generating DestinyPlaceDefinition dictionary....
Generating DestinyActivityDefinition dictionary....
Generating DestinyActivityTypeDefinition dictionary....
Generating DestinyClassDefinition dictionary....
Generating DestinyGenderDefinition dictionary....
Generating DestinyInventoryBucketDefinition dictionary....
Generating DestinyRaceDefinition dictionary....
Generating DestinyTalentGridDefinition dictionary....
Generating DestinyUnlockDefinition dictionary....
Generating DestinyMaterialRequirementSetDefinition dictionary....
Generating DestinySandboxPerkDefinition dictionary....
Generating DestinyStatGroupDefinition dictionary....
Generating DestinyFactionDefinition dictionary....
Generating DestinyVendorGroupDefinition dictionary....
Generating DestinyRewardSourceDefinition dictionary....
Generating DestinyItemCategoryDefinition dictionary....
Generating DestinyDamageTypeDefinition dictionary....
Generating DestinyActivityModeDefinition dictionary....
Generating DestinyMed

In [21]:
# all_data # DON'T RUN THIS, YOU'LL LAG OUT THE NOTEBOOK

In [34]:
list(all_data['DestinyInventoryItemDefinition'].items())[4]  # THIS IS THE ONE FOR ITEMS, BOTH ARMOR AND WEAPONS

In [35]:
c42_hash = 2149166938  # hash for Classical-42, some random weapon

In [36]:
len(all_data['DestinyInventoryItemDefinition'])  # number of items in the game I guess

16720

In [52]:
all_data['DestinyInventoryItemDefinition'][c42_hash]['hash'] == c42_hash  # you can get the hash from the hash lul
# this means we can potentially reformat to make the name of the weapon the key,
# since the hash itself is still available in the data

True

In [58]:
#all_data['DestinyInventoryItemDefinition'][c42_hash]

In [57]:
#let's try searching for a weapon
desired = "Annual Skate"
results = []
for item in list(all_data['DestinyInventoryItemDefinition'].items()):
    #print(item[1]['displayProperties']['name'])
    if item[1]['displayProperties']['name'] == desired:
        print('found one')
        results.append(item)

found one
found one


In [76]:
results
#results[0][1]#.keys()

# pretty print
print(json.dumps(results[0][1], indent=4))

{
    "displayProperties": {
        "description": "",
        "name": "Annual Skate",
        "icon": "/common/destiny2_content/icons/f288cca4a874307fff1e57bc2a295f38.jpg",
        "hasIcon": true
    },
    "tooltipNotifications": [],
    "collectibleHash": 579156770,
    "iconWatermark": "/common/destiny2_content/icons/4368a3e344977c5551407845ede830c2.png",
    "secondaryIcon": "/common/destiny2_content/icons/e9fcd73e969a9295c3ab4ee5743893c2.png",
    "backgroundColor": {
        "colorHash": 0,
        "red": 0,
        "green": 0,
        "blue": 0,
        "alpha": 0
    },
    "screenshot": "/common/destiny2_content/screenshots/6857689.jpg",
    "itemTypeDisplayName": "Hand Cannon",
    "flavorText": "\"When the lake used to freeze, people would dance on the ice wearing boots with tiny swords. And fall in love.\"",
    "uiItemDisplayStyle": "",
    "itemTypeAndTierDisplayName": "Legendary Hand Cannon",
    "displaySource": "Random Perks: This item cannot be reacquired from Coll

In [84]:
print(results[0][1]['quality']['versions'][0]['powerCapHash'])  # EZ PZ
print(results[1][1]['quality']['versions'][0]['powerCapHash'])  # EZ PZ

2759499571
1862490583


In [61]:
# 
print(all_data['DestinyPowerCapDefinition'][1862490583]) # CAPPED ANNUAL SKATE
print(all_data['DestinyPowerCapDefinition'][2759499571]) # UNCAPPED ANNUAL SKATE

{'powerCap': 1060, 'hash': 1862490583, 'index': 7, 'redacted': False, 'blacklisted': False}
{'powerCap': 999990, 'hash': 2759499571, 'index': 0, 'redacted': False, 'blacklisted': False}


## Okay NICE
#### Now let's make a data structure for ideally just weapons with the names as keys instead of hash
#### so key: name, value: list of items from all_data['DestinyInventoryItemDefinition'] (item can show up multiple times, i.e. if sunset and un-sunset maybe
#### Potential issues with something like aachen as well, where it exists as both kinetic and energy
#### For now, just make the data structure's values a list of possible correlations to the name.  We can handle which one is correct later on.

In [99]:
results[0][1]['inventory']['bucketTypeHash']  # item type hash

2465295065

In [101]:
all_data['DestinyInventoryBucketDefinition'][2465295065]  # refers to Energy Weapon hash (OKAY!)

{'displayProperties': {'description': 'Weapons that deal Arc, Solar, or Void damage. Most effective when dealing with shielded enemies.',
  'name': 'Energy Weapons',
  'hasIcon': False},
 'scope': 0,
 'category': 3,
 'bucketOrder': 30,
 'itemCount': 10,
 'location': 1,
 'hasTransferDestination': True,
 'enabled': True,
 'fifo': False,
 'hash': 2465295065,
 'index': 1,
 'redacted': False,
 'blacklisted': False}

In [120]:
# get bucket hashes for weapons
weapon_buckets = {} # lets do {buckethash, name} as the format
for item in list(all_data['DestinyInventoryBucketDefinition'].items()):
    if "name" in item[1]['displayProperties'].keys():
        if "weapon" in item[1]['displayProperties']['name'].lower():
            weapon_buckets[item[0]] = item[1]['displayProperties']['name']
            
print(weapon_buckets)
# now we can verify an item is a wepon by checking if item['inventory']['bucketTypeHash'] is in weapon_bucket.keys() !

{2465295065: 'Energy Weapons', 953998645: 'Power Weapons', 1498876634: 'Kinetic Weapons'}


In [126]:
list(all_data['DestinyInventoryItemDefinition'].items())[4][1]['inventory']['bucketTypeHash']

953998645

In [128]:
list(all_data['DestinyInventoryItemDefinition'].items())[4][1]['displayProperties']['name']

'Classical-42'

In [195]:
# let's build this thing
# keys are weapon names
weapons_dict = {}
for item in list(all_data['DestinyInventoryItemDefinition'].items()):
    
    # check if weapon and 'quality' (used for checking light level) is available
    if (item[1]['inventory']['bucketTypeHash'] in weapon_buckets.keys()) and ('quality' in item[1].keys()):
        name = item[1]['displayProperties']['name']
        
        # if not yet in weapons_dict:
        if name not in weapons_dict.keys():
            weapons_dict[name] = []
        weapons_dict[name].append(item[1])


# show some output
for weapon in list(weapons_dict.items())[:3]:
    print(json.dumps(weapon, indent=4))
    print("\n")

[
    "Classical-42",
    [
        {
            "displayProperties": {
                "description": "",
                "name": "Classical-42",
                "icon": "/common/destiny2_content/icons/ba7c34b68fc4292a88dd4fa46aa54a85.jpg",
                "hasIcon": true
            },
            "tooltipNotifications": [],
            "collectibleHash": 1551165456,
            "iconWatermark": "/common/destiny2_content/icons/591f14483308beaad3278c3cd397e284.png",
            "iconWatermarkShelved": "/common/destiny2_content/icons/50d36366595897d49b5d33e101c8fd07.png",
            "secondaryIcon": "/common/destiny2_content/icons/e9fcd73e969a9295c3ab4ee5743893c2.png",
            "backgroundColor": {
                "colorHash": 0,
                "red": 0,
                "green": 0,
                "blue": 0,
                "alpha": 0
            },
            "screenshot": "/common/destiny2_content/screenshots/2149166938.jpg",
            "itemTypeDisplayName": "Rocket Launcher

In [180]:
len(weapons_dict["Annual Skate"]) # one sunset, one non-sunset

2

#### Okay.  Now let's move into modules and incorporate with discord bot.  VERY good progress

In [181]:
print(results[0][1]['quality']['versions'][0]['powerCapHash'])  # EZ PZ
print(results[1][1]['quality']['versions'][0]['powerCapHash'])  # EZ PZ

print(all_data['DestinyPowerCapDefinition'][1862490583]) # CAPPED ANNUAL SKATE
print(all_data['DestinyPowerCapDefinition'][2759499571]) # UNCAPPED ANNUAL SKATE

2759499571
1862490583
{'powerCap': 1060, 'hash': 1862490583, 'index': 7, 'redacted': False, 'blacklisted': False}
{'powerCap': 999990, 'hash': 2759499571, 'index': 0, 'redacted': False, 'blacklisted': False}


In [207]:
print(all_data['DestinyPowerCapDefinition'][weapons_dict['Annual Skate'][0]['quality']['versions'][0]['powerCapHash']]['powerCap'])
print(all_data['DestinyPowerCapDefinition'][weapons_dict['Annual Skate'][1]['quality']['versions'][0]['powerCapHash']]['powerCap'])

1060
999990


In [210]:
# sort weapons_dict by power level cap
# that way when users want to get item, we just return info from the first in the list
# i.e. if a weapon is listed twice, the one with higher power cap will be returned (un-sunset version)
for key, val in weapons_dict.items():
    weapons_dict[key] = sorted(val, key = lambda x: all_data['DestinyPowerCapDefinition'][x['quality']['versions'][0]['powerCapHash']]['powerCap'], reverse=True)
        

In [176]:
len(weapons_dict['Fatebringer'])

2

In [187]:
weapons_dict['Fatebringer'][0] == weapons_dict['Fatebringer'][1]

False

[1, 3, 2]